In [1]:
from gen_model2 import gen_model, load_data

In [5]:
from tqdm import tqdm
from collections import defaultdict
from pathlib import Path

In [7]:
export_path = Path('2019-01-08-event_data')

export_path / "hola"

PosixPath('2019-01-08-event_data/hola')

In [2]:
models = {
    'libya': gen_model('libya_hotel_tweets.tsv'),
    'pistorius': gen_model('oscar_pistorius_tweets.tsv'),
    'nepal': gen_model('nepal_tweets.tsv')
}

100%|██████████| 503660/503660 [00:00<00:00, 898550.56it/s]


# Event data

## tweet ids, texts, replies, rts

In [14]:
with (export_path / 'event_data.tsv').open('w') as f:
    f.write('event\ttweet_id\tretweet_id\treply_id\ttext\n')
    for event, info in tqdm(models.items()):
        for tweet_id, tweet in info['event_data'].items():
            text = tweet.text
            text = text.replace('"', "'")
            text = text.split()
            text = ' '.join(text)
            text = f'"{text}"'
            
            rt_id = tweet.retweet_id if tweet.retweet_id != "NULL" else "NA"
            rp_id = tweet.reply_id if tweet.reply_id != "NULL" else "NA"
            
            f.write(f'{event}\t{tweet_id}\t{rt_id}\t{rp_id}\t{text}\n')

100%|██████████| 3/3 [00:01<00:00,  1.95it/s]


## urls for each tweet

In [11]:
# example:
models['libya']['event_data']['560028495792050176']

Tweet(tweet_id='560028495792050176', retweet_id='560027393432502272', quote_id='NULL', reply_id='NULL', short_urls=['http://t.co/5TK7TsFd6P', 'http://t.co/jszG3p'], expanded_urls={0: 'https://www.rt.com/news/226603-libya-tripoli-gunmen-seige/', 1: None}, text='#CorinthiaHotel: Suicide bombers &amp; 5 masked gunmen attack hotel, hostages on top floor  http://t.co/5TK7TsFd6P #Lybia http://t.co/jszG3p', created_at='2015-01-27 10:56:12')

In [13]:
with (export_path / 'event_data_urls.tsv').open('w') as f:
    f.write('event\ttweet_id\turl\n')
    for event, info in tqdm(models.items()):
        for tweet_id, tweet in info['event_data'].items():
            expanded_urls = tweet.expanded_urls
            
            for url in expanded_urls.values():
                if url:
                    f.write(f'{event}\t{tweet_id}\t{url}\n')

100%|██████████| 3/3 [00:00<00:00,  6.42it/s]


# Model data